In [1]:
import numpy as np
import scipy as sp
from scipy import linalg as la

In [2]:
#Problem 1
def modified_GS (A):
    A = np.array(A)
    m , n = A.shape
    Q = np.copy(A)
    R = np.zeros((n, n))
    for i in np.arange(n):
        R[i, i] = la.norm(Q[:, i])
        Q[:, i] = Q[:, i]*(1/R[i, i])
        for j in np.arange((i+1), n):
            R[i, j] = np.dot(np.transpose(Q[:, j]),Q[:, i])
            Q[:, j] = Q[:, j] - R[i, j]*Q[:, i]
    return Q, R
    

A = np.random.random((6,4))
# my function
Q_my, R_my = modified_GS(A)
print("My QR decomposition: \n")
print(Q_my, R_my)
#scipy's function
Q_sp,R_sp = la.qr(A, mode="economic")
print("Scipy's QR decomposition: \n")
print(Q_sp, R_sp)
print("The results might be different since QR decomposition is not unique")
print (A.shape, Q_my.shape, R_my.shape)
print(np.allclose(np.triu(R_my), R_my))
print(np.allclose(np.dot(Q_my.T, Q_my), np.identity(4)))
print(np.allclose(np.dot(Q_my, R_my), A))

My QR decomposition: 

[[ 0.50658975  0.13626751 -0.41834406 -0.09400699]
 [ 0.15579745  0.89131504  0.19771933  0.3545368 ]
 [ 0.50386541 -0.09922196 -0.55557498  0.15448913]
 [ 0.35335349  0.05936637  0.41564855 -0.44583654]
 [ 0.43984169 -0.04580842  0.32709999 -0.42727979]
 [ 0.38326783 -0.41415081  0.44438006  0.67842554]] [[ 1.90054453  0.7393178   1.36836033  0.61736669]
 [ 0.          0.58014298  0.40641393  0.35405742]
 [ 0.          0.          0.93645044  0.48347854]
 [ 0.          0.          0.          0.6485544 ]]
Scipy's QR decomposition: 

[[-0.50658975 -0.13626751 -0.41834406 -0.09400699]
 [-0.15579745 -0.89131504  0.19771933  0.3545368 ]
 [-0.50386541  0.09922196 -0.55557498  0.15448913]
 [-0.35335349 -0.05936637  0.41564855 -0.44583654]
 [-0.43984169  0.04580842  0.32709999 -0.42727979]
 [-0.38326783  0.41415081  0.44438006  0.67842554]] [[-1.90054453 -0.7393178  -1.36836033 -0.61736669]
 [ 0.         -0.58014298 -0.40641393 -0.35405742]
 [ 0.          0.          0

In [3]:
#Problem 2
M_init = np.random.random((2,2))
QRdet = lambda M: abs(np.diag(modified_GS(M)[1]).prod())
print(QRdet(M_init))

0.0616431935324


In [4]:
#problem 3
def QRlin_sol(A, b):
    A = np.array(A)
    b = np.array(b)
    n = A.shape[0]
    Q, R = modified_GS(A)
    y = np.transpose(Q)@b
    x = np.zeros_like(b)
    for i in np.arange(n)[::-1]:
        x[i] = (1/R[i, i])*(y[i] - np.transpose(x)@R[i, :])
    return x

#testing on my function
A = [[1, 1], [0, 1]]
b = [2, 3]
QRlin_sol(A, b)

array([-1,  3])

In [5]:
#problem 4
def householderQR (A):
    A = np.array(A)
    m, n = A.shape
    R = np.copy(A)
    Q = np.identity(m)
    sign = lambda x: 1 if x >= 0 else -1
    for k in range(n):
        u = np.copy(R[k:, k])
        u[0] = u[0] + sign(u[0])*la.norm(u)
        u  = u*(1/la.norm(u))
        R[k:, k:] = R[k:, k:] - 2*np.outer(u, np.dot(u, R[k:, k:]))
        Q[k:, :] = Q[k:, :] - 2*np.outer(u, np.dot(u, Q[k:, :]))
        
    return np.transpose(Q), R
# testing on the funciton of household transformation
A = np.random.random((5, 3))
Q, R = householderQR(A)
print(A.shape, Q.shape, R.shape)
np.allclose(np.dot(Q, R), A)

(5, 3) (5, 5) (5, 3)


True

In [7]:
# problem 5
def hessenberg(A):
    m, n  = A.shape
    H = np.copy(A)
    Q = np.identity(m)
    sign = lambda x: 1 if x >= 0 else -1
    for k in np.arange(n-2):
        u = np.copy(H[(k+1):, k])
        u[0] = u[0] + sign(u[0])*la.norm(u)
        u  = u*(1/la.norm(u))
        H[(k+1):,k: ] = H[(k+1):,k: ] - 2*np.outer(u, np.dot(u, H[(k+1):,k: ]))
        H[:, (k+1):] = H[:, (k+1):] - 2*np.outer(np.dot(H[:, (k+1):] , u), u)
        Q[(k+1):, :] = Q[(k+1):, :] -2*np.outer(u, np.dot(u, Q[(k+1):, :]))
    
    return H, np.transpose(Q)

A = np.random.random((8,8))
H_my, Q_my = hessenberg(A)
H, Q = la.hessenberg(A, calc_q=True)

print(np.allclose(np.triu(H, -1), H))
print(np.allclose(np.dot(np.dot(Q, H), Q.T), A))

print(np.allclose(np.triu(H_my, -1), H_my))
print(np.allclose(np.dot(np.dot(Q_my, H_my), Q_my.T), A))


True
True
True
True
